# Prepare Development Environment

This will create the virtual environment

In [ ]:
!conda create -c pytorch -c nvidia -n gen_voice pytorch torchvision torchaudio pytorch-cuda=11.8 -y
!conda activate gen_voice
!conda install conda-forge::ffmpeg
!pip install -r requirements.txt

If ffmpeg doesn't work correctly on Linux the following command may help

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

# Import Libraries

In [1]:
from chatbot import ChatBot
import speech_recognition as sr
import gradio as gr

%load_ext gradio

c:\Users\holes\.conda\envs\ik\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Initialize the ChatBot

First check for available microphones

In [ ]:
# List the available microphones
sr.Microphone.list_microphone_names()

## Full Audio
If you're system has speakers and a microphone you can run in full audio using the communication loop below

In [5]:
# Initialize the chatbot specifying the index of the mic if one exists and whether or not speakers will be used.
# For best results a headset with a mic is recommended.
chatbot = ChatBot(mic_id=2, enable_speakers=True)

history = []
while True:
    guess = chatbot.recognize_speech_from_mic()
    if guess["success"]:
        prompt = guess['transcription']
        print(f'Prompt: {prompt} \n')

        if 'goodbye' in prompt:
            chatbot.communicate('See you next time')
            break
        
        response = chatbot.respond(prompt, history)
        history.append([guess['transcription'], response])
        print(f'{response}\n\n')
    else:
        chatbot.communicate(f"Error: {guess["error"]}")

Prompt: can I get a small pepperoni pizza 

Sure! A small pepperoni pizza is $7.00. Would you like to add any extra toppings to that?


Prompt: no 

Got it! Just the small pepperoni pizza. Would you like anything else with your order, maybe some drinks or a side of fries?


Prompt: goodbye 



## Speakers Optional (no mic)
The following gradio UI will allow you to communicate with the LLM.

In [3]:
%%blocks

#import gradio as gr
# Create a gradio chat interface wire it to our chatbot
chatbot = ChatBot(enable_speakers=True)
with gr.Blocks() as demo:
    gr.ChatInterface(chatbot.respond)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
